In [7]:
# define function
def genSankey(df,cat_cols=[],value_cols='',title='Sankey Diagram'):
    # maximum of 6 value cols -> 6 colors
    colorPalette = ['#1E88E5','#D81B60','#FFC107',] # colorblind palette
    labelList = []
    colorNumList = []
    for catCol in cat_cols:
        labelListTemp =  list(set(df[catCol].values))
        colorNumList.append(len(labelListTemp))
        labelList = labelList + labelListTemp
        
    # remove duplicates from labelList
    labelList = list(dict.fromkeys(labelList))
    
    # define colors based on number of levels
    colorList = []
    for idx, colorNum in enumerate(colorNumList):
        colorList = colorList + [colorPalette[idx]]*colorNum
        
    # transform df into a source-target pair
    for i in range(len(cat_cols)-1):
        if i==0:
            sourceTargetDf = df[[cat_cols[i],cat_cols[i+1],value_cols]]
            sourceTargetDf.columns = ['source','target','count']
        else:
            tempDf = df[[cat_cols[i],cat_cols[i+1],value_cols]]
            tempDf.columns = ['source','target','count']
            sourceTargetDf = pd.concat([sourceTargetDf,tempDf])
        sourceTargetDf = sourceTargetDf.groupby(['source','target']).agg({'count':'sum'}).reset_index()
        
    # add index for source-target pair
    sourceTargetDf['sourceID'] = sourceTargetDf['source'].apply(lambda x: labelList.index(x))
    sourceTargetDf['targetID'] = sourceTargetDf['target'].apply(lambda x: labelList.index(x))
    
    # creating the sankey diagram
    data = dict(
        type='sankey',
        node = dict(
          pad = 15,
          thickness = 20,
          line = dict(
            color = "black",
            width = 0.5
          ),
          label = labelList,
          color = colorList
        ),
        link = dict(
          source = sourceTargetDf['sourceID'],
          target = sourceTargetDf['targetID'],
          value = sourceTargetDf['count']
        )
      )
    
    layout =  dict(
        title = title,
        font = dict(
          size = 10
        )
    )
       
    fig = dict(data=[data], layout=layout)
    return fig

In [8]:
# import packages
import pandas as pd
import plotly
import plotly as py



In [9]:
# read in dataframe and subset dataframe
df = pd.read_csv("postgradcareers.csv")

In [5]:
# check df
df.head()

,lvl1,lvl2,lvl3,count
0,PhD/Professional degree in Psychology,Social & Related Scientists,Psychologist,92600
1,PhD/Professional degree in Psychology,Social & Related Scientists,Postsecondary teacher: psychology & other,26600
2,PhD/Professional degree in Psychology,Social & Related Scientists,Other social scientists,7300
3,PhD/Professional degree in Psychology,Science & Engineering Related Occupations,Medical and health services managers,7200
4,PhD/Professional degree in Psychology,Science & Engineering Related Occupations,Diagnosing/treating practitioner,3200


In [10]:
# make sankey diagram
fig = genSankey(df,cat_cols=['lvl1','lvl2','lvl3'],value_cols='count',title='Postgraduate Careerpaths')
plotly.offline.plot(fig, validate=False)

'temp-plot.html'

In [ ]:
# source data 
#https://www.apa.org/workforce/data-tools/careers-psychology

In [17]:
# save as vector diagram for further editing in illustrator
fig = genSankey(df,cat_cols=['lvl1','lvl2','lvl3'],value_cols='count',title='Postgraduate Careerpaths')
plotly.offline.plot(fig, validate=False, image_filename='postgrad_career', image='svg', image_width = 1900, image_height=1900)

'temp-plot.html'